In [5]:
import pandas as pd
import numpy as np
import talib
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from imp import reload
import src.strats as strats
reload(strats)
import src.strats
import src.helpers as helpers



In [634]:
df = pd.read_csv('../data/clean/swe_equ/SBB-B.csv', parse_dates=['Date'], index_col=['Date'])
states = pd.Series(index = np.load('louvain_state_i.npy', allow_pickle=True), data = np.load('louvain_state.npy', allow_pickle=True))
# Add cols
df = df.join(states.rename('states'))
df['ma200'] = talib.SMA(df.Close)
df['rsi'] = talib.RSI(df.Close)
df['ma50'] = talib.SMA(df.Close, timeperiod=50)
df['rsi3'] = talib.RSI(df.Close, timeperiod=3)
_, _, df['bol_lower'] = talib.BBANDS(df.Close)

# add Y column
df['Y'] = np.log(df.Close.shift(-5)) - np.log(df.Close) # what is the return, x days from now?
df['Y'] = df['Y'].apply(lambda x: 0 if x < 0 else 2 if x > 0.5 else 1)# else 3 if x > 0.1 else 2 if x > 0.05 else 1)

# Returns
df.Open = np.log(df.Open).diff()
df.High = np.log(df.High).diff()
df.Low = np.log(df.Low).diff()
df.Close = np.log(df.Close).diff()

# Shifting phase
for i in range(20):
    df['state'+str(i)] = df.states.shift(i)
    df['rsi'+str(i)] = df.rsi.shift(i)
    df['close'+str(i)] = df.Close.shift(i)

# NOW ADDING SIGNALS
roll_data = df.iloc[::-1]
df['mom_signal'] = helpers.roll(roll_data, 40).apply(strats.momentum_strat)
df['revert_signal'] = helpers.roll(roll_data, 5).apply(strats.mean_revert_strat)

# drop all NaN
df.dropna(inplace=True)
df.shape

C:\Users\agato\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\agato\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\agato\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\agato\Documents\FBD\EPFL-financial-big-data\ta_method\strats.py:77: RuntimeWarning: divide by zero encountered in double_scalars
  price_slope = price_slope/df.Close[0]*100
C:\Users\agato\Documents\FBD\EPFL-financial-big-data\ta_method\strats.py:79: RuntimeWarning: divide by zero encountered in double_scalars
  ma200_slope = ma200_slope/df.Close[0]*100
C:\Users\agato\Documents\FBD\EPFL-financial-big-data\ta_method\strats.py:78: RuntimeWarning: divide by zero encountered in double_scalars
 

(1241, 73)

In [647]:
clf = RandomForestClassifier()
train = df.iloc[0:1000,:]
test = df.iloc[1000:,:]

drop_cols = ['Open', 'High', 'Low', 'ma200', 'rsi','Y', 'ma50', 'rsi3', 'bol_lower']

x_test = test.drop(drop_cols, axis=1)
y_test = test.Y

x_train = train.drop(drop_cols, axis=1)
y_train = train.Y

clf.fit(x_train, y_train)
print(x_train.columns)
print(clf.feature_importances_)
prediction = clf.predict(x_test)
print(clf.score(x_test, y_test))
#clf.decision_path(x_test)

test['prediction'] = prediction
print('\n1, actual vs prediction')
print(test[test['Y'] == 1].shape[0], test[test['prediction'] == 1].shape[0])
print('\n0, actual vs prediction')
print(test[test['Y'] == 0].shape[0], test[test['prediction'] == 0].shape[0])
print('\n2, actual vs prediction')
print(test[test['Y'] == 2].shape[0], test[test['prediction'] == 2].shape[0])

test[test['prediction'] == 2]

Index(['Close', 'Volume', 'states', 'state0', 'rsi0', 'close0', 'state1',
       'rsi1', 'close1', 'state2', 'rsi2', 'close2', 'state3', 'close3',
       'state4', 'rsi4', 'close4', 'state5', 'rsi5', 'close5', 'state6',
       'rsi6', 'close6', 'state7', 'rsi7', 'close7', 'state8', 'rsi8',
       'close8', 'state9', 'rsi9', 'close9', 'state10', 'rsi10', 'close10',
       'state11', 'rsi11', 'close11', 'state12', 'rsi12', 'close12', 'state13',
       'rsi13', 'close13', 'state14', 'rsi14', 'close14', 'state15', 'rsi15',
       'close15', 'state16', 'rsi16', 'close16', 'state17', 'rsi17', 'close17',
       'state18', 'rsi18', 'close18', 'state19', 'rsi19', 'close19',
       'mom_signal', 'revert_signal'],
      dtype='object')
[0.0193043  0.01627197 0.00536443 0.00650759 0.0266865  0.00999431
 0.007117   0.02064563 0.02250826 0.00804175 0.02073658 0.0187296
 0.01244333 0.01715758 0.01156403 0.01554018 0.01317352 0.01405058
 0.01711267 0.01896936 0.0072216  0.02161079 0.02684947 0.0144412

C:\Users\agato\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\agato\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Open,High,Low,Close,Volume,states,ma200,rsi,ma50,rsi3,...,close17,state18,rsi18,close18,state19,rsi19,close19,mom_signal,revert_signal,prediction
Date,,,,,,,,,,,,,,,,,,,,,
